In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

c:\Users\ruben\anaconda3\envs\rag\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'fd4f12f50ec30297683cb4eed799e312', 'Date': 'Tue, 27 Aug 2024 15:06:27 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [3]:
import json
data = json.load(open('reviews.json'))

In [4]:
processed_data = []
client = OpenAI()

for review in data["reviews"]:
    response = client.embeddings.create(
        input=review["review"],
        model="text-embedding-3-small",

    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "subject": review["subject"],
            "review": review["review"],
            "stars": review["stars"]
        }
    })

In [5]:
processed_data[0]

{'values': [-0.020624535158276558,
  -0.019661569967865944,
  0.011829853057861328,
  0.045612890273332596,
  0.02061234600841999,
  0.016772672533988953,
  -0.00850213598459959,
  0.015919411554932594,
  -0.00445219362154603,
  -0.0117750009521842,
  0.029084008187055588,
  -0.02759689651429653,
  0.00563152227550745,
  -0.025987889617681503,
  -0.019466537982225418,
  0.03985947370529175,
  -0.014420109800994396,
  0.016004737466573715,
  0.03179006278514862,
  0.056120190769433975,
  0.016857998445630074,
  -0.02920590341091156,
  0.004769118968397379,
  -0.014261647127568722,
  -0.05948447436094284,
  -0.0634826123714447,
  0.023988820612430573,
  0.025183387100696564,
  -0.0024287463165819645,
  0.010239130817353725,
  0.07484317570924759,
  -0.009568711742758751,
  -0.014346973039209843,
  -0.023842548951506615,
  -0.05753416568040848,
  0.02195318415760994,
  0.0011709482641890645,
  0.05987453833222389,
  0.013249923475086689,
  0.008940955623984337,
  0.021087734028697014,
  -

In [6]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 50}

In [7]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}